In [11]:
import pandas as pd
import requests
import plotly.express as px

In [12]:
# Use the correct base URL for Binance Futures API (fapi.binance.com)
url = f"https://fapi.binance.com/fapi/v1/klines"
params = {
    "symbol": "SOLUSDT",
    "interval": "1m",
    "limit": 100
}

try:
    response = requests.get(url, params=params)
    response.raise_for_status() # Raises an HTTPError for bad responses (4xx or 5xx)
    data = response.json()
    print(data)
except requests.exceptions.RequestException as e:
    print(f"Request failed: {e}")
    if 'response' in locals():
        print(f"Response text: {response.text}")

[[1764569640000, '126.3900', '126.4600', '126.3800', '126.4300', '18073.22', 1764569699999, '2284757.371700', 1393, '5191.40', '656305.361800', '0'], [1764569700000, '126.4300', '126.6000', '126.3600', '126.5300', '26521.23', 1764569759999, '3354605.303600', 3403, '17290.46', '2187050.208200', '0'], [1764569760000, '126.5300', '126.5300', '126.3300', '126.3400', '25373.20', 1764569819999, '3207881.742500', 1495, '4299.83', '543641.449700', '0'], [1764569820000, '126.3300', '126.4800', '126.3000', '126.3200', '11096.44', 1764569879999, '1402730.126600', 1546, '6665.73', '842666.297400', '0'], [1764569880000, '126.3200', '126.4300', '126.3200', '126.3900', '5030.07', 1764569939999, '635578.980100', 988, '3952.59', '499434.172200', '0'], [1764569940000, '126.3800', '126.3800', '126.2200', '126.2300', '4075.39', 1764569999999, '514726.119400', 1106, '2123.88', '268220.083600', '0'], [1764570000000, '126.2300', '126.2300', '126.1400', '126.1400', '6816.12', 1764570059999, '860042.943800', 1

In [13]:
df_raw = pd.DataFrame(data)
df_raw.columns = [
    "open_time", "open", "high", "low", "close", "volume",
    "close_time", "quote_asset_volume", "number_of_trades",
    "taker_buy_base_asset_volume", "taker_buy_quote_asset_volume", "ignore"
]
df_raw

,open_time,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,ignore
0,1764569640000,126.3900,126.4600,126.3800,126.4300,18073.22,1764569699999,2284757.371700,1393,5191.40,656305.361800,0
1,1764569700000,126.4300,126.6000,126.3600,126.5300,26521.23,1764569759999,3354605.303600,3403,17290.46,2187050.208200,0
2,1764569760000,126.5300,126.5300,126.3300,126.3400,25373.20,1764569819999,3207881.742500,1495,4299.83,543641.449700,0
3,1764569820000,126.3300,126.4800,126.3000,126.3200,11096.44,1764569879999,1402730.126600,1546,6665.73,842666.297400,0
4,1764569880000,126.3200,126.4300,126.3200,126.3900,5030.07,1764569939999,635578.980100,988,3952.59,499434.172200,0
...,...,...,...,...,...,...,...,...,...,...,...,...
95,1764575340000,127.0200,127.1700,126.9900,127.1700,13922.83,1764575399999,1769447.906700,1707,10016.15,1272921.497000,0
96,1764575400000,127.1700,127.1900,127.0400,127.0500,16123.32,1764575459999,2049680.594200,1727,6066.51,771170.131700,0
97,1764575460000,127.0600,127.1900,127.0000,127.1700,14665.96,1764575519999,1864104.778800,1619,8441.58,1072892.587700,0
98,1764575520000,127.1700,127.2800,127.1600,127.2600,13780.70,1764575579999,1753447.573200,1541,9753.74,1241022.598600,0


In [14]:
# --- DataFrame 1: Opening Data ---
df_open = df_raw[['open_time', 'open']].copy() 
df_open.columns = ['opening_time', 'opening_price']

# 1. Convert 'opening_time' from Unix timestamp (ms) to readable datetime objects
# 2. Convert 'opening_price' from string to float for numerical analysis
df_open['opening_time'] = pd.to_datetime(df_open['opening_time'], unit='ms')
df_open['opening_price'] = df_open['opening_price'].astype(float)
df_open

,opening_time,opening_price
0,2025-12-01 06:14:00,126.39
1,2025-12-01 06:15:00,126.43
2,2025-12-01 06:16:00,126.53
3,2025-12-01 06:17:00,126.33
4,2025-12-01 06:18:00,126.32
...,...,...
95,2025-12-01 07:49:00,127.02
96,2025-12-01 07:50:00,127.17
97,2025-12-01 07:51:00,127.06
98,2025-12-01 07:52:00,127.17


In [15]:
# --- DataFrame 2: Closing Data ---
df_close = df_raw[['close_time', 'close']].copy()
df_close.columns = ['closing_time', 'closing_price']

# Data Type Conversion:
# 1. Convert 'closing_time' from Unix timestamp (ms) to readable datetime objects
# 2. Convert 'closing_price' from string to float
df_close['closing_time'] = pd.to_datetime(df_close['closing_time'], unit='ms')
df_close['closing_price'] = df_close['closing_price'].astype(float)
df_close

,closing_time,closing_price
0,2025-12-01 06:14:59.999,126.43
1,2025-12-01 06:15:59.999,126.53
2,2025-12-01 06:16:59.999,126.34
3,2025-12-01 06:17:59.999,126.32
4,2025-12-01 06:18:59.999,126.39
...,...,...
95,2025-12-01 07:49:59.999,127.17
96,2025-12-01 07:50:59.999,127.05
97,2025-12-01 07:51:59.999,127.17
98,2025-12-01 07:52:59.999,127.26


## Using Plotly

In [16]:
# line chart for opening prices
fig = px.line(
    df_open,
    x='opening_time',
    y='opening_price',
    title='Line Chart of Opening Prices Over Time'
)
fig.show()

In [17]:
# line chart for closing prices
fig = px.line(
    df_close,
    x='closing_time',
    y='closing_price',
    title='Line Chart of Closing Prices Over Time'
)
fig.show()

In [18]:
# Rename time columns for clarity before merging
df_open_merge = df_open.rename(columns={'opening_time': 'time'})
df_close_merge = df_close.rename(columns={'closing_time': 'close_time'})

# Merge the two DataFrames
# CRITICAL: We merge on the index (left_index=True, right_index=True).
# We cannot merge on 'time' because the Open Time and Close Time of the same candle are different.
# By merging on the index, we ensure that the Open and Close prices for the *same candle* are aligned.
df_combined = pd.merge(
    df_open_merge[['time', 'opening_price']],
    df_close_merge[['closing_price']],
    left_index=True,
    right_index=True,
    how='inner'
)

df_combined_long = pd.melt(
    df_combined,
    id_vars=['time'],
    value_vars=['opening_price', 'closing_price'],
    var_name='Price_Type',      # New column for the label (e.g., 'opening_price')
    value_name='Price_Value'    # New column for the data (e.g., 45000)
)

df_combined_long

,time,Price_Type,Price_Value
0,2025-12-01 06:14:00,opening_price,126.39
1,2025-12-01 06:15:00,opening_price,126.43
2,2025-12-01 06:16:00,opening_price,126.53
3,2025-12-01 06:17:00,opening_price,126.33
4,2025-12-01 06:18:00,opening_price,126.32
...,...,...,...
195,2025-12-01 07:49:00,closing_price,127.17
196,2025-12-01 07:50:00,closing_price,127.05
197,2025-12-01 07:51:00,closing_price,127.17
198,2025-12-01 07:52:00,closing_price,127.26


In [19]:
fig = px.line(
    df_combined_long,
    x='time',          
    y='Price_Value',
    color='Price_Type',
    title='Opening vs. Closing Prices Over Time'
)

# Customize the labels displayed in the legend/tooltip
fig.for_each_trace(lambda t: t.update(name = t.name.replace('_', ' ').title()))

fig.update_layout(
    xaxis_title='Time',
    yaxis_title='Price',
    hovermode='x unified'
)

fig.show()